### Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

#### **Let's start by gathering the MCP params for our trader**

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [3]:
#mcps for trader

if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},   #account mcp server that we created 
    {"command": "uv", "args": ["run", "push_server.py"]}, #push mcp server for push notification 
    market_mcp
]

In [4]:
#mcp for researcher 

serp_env = {"SERPER_API_KEY": os.getenv("SERPER_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "uvx", "args": ["serper-mcp-server"], "env": serp_env}
]

In [5]:
#now create MCPServerStudio for each 

researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]  #research servers 
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params] #trade servers 
mcp_servers = trader_mcp_servers + researcher_mcp_servers


In [6]:
for server in mcp_servers:
    try:
        await server.close()
    except:
        pass

#### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [ ]:

async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [17]:
#lets test it directly before integrating as a tool 

research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()     #alternative way for the context manager  i.e. (async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:)
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)  #max_turns tells how many time the agent can call the tools
display(Markdown(result.final_output))


Here are the latest news highlights on Amazon:

1. Amazon is ending the free shipping perk for its Prime Invitee program, which allowed Prime members to share free shipping with others. This change has been reported by USA Today, Investopedia, and The New York Times over the past few days.

2. Amazon plans to shutter a popular free app by late 2024 to focus streaming efforts on Amazon Prime, according to AL.com.

3. The Amazon VP of Global Video Advertising, Krishan Bhatia, left the company recently after joining in April 2024, as reported by ADWEEK.

4. Amazon is promoting stylish fall outfits with a range of fashionable items at various price points, noted by Travel + Leisure and Real Simple.

5. There is a nationwide recall of thousands of baby loungers sold on Amazon due to a risk of death warning from the U.S. Consumer Product Safety Commission, according to Newsweek.

6. Amazon-backed AI startup Fable announced an ambitious project related to Orson Welles' fan fiction, covered by TechCrunch.

7. A Long Island man allegedly sold thousands of fake Nintendo products via Amazon for years, reported by NBC New York.

If you want, I can provide more details on any specific news or investigate trading opportunities based on these developments.

In [19]:
#integrating the research agent as a tool 

async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [21]:
#changing the marketing stategy of a person 

ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ed_initial_strategy)

#these two are the resources, the openAI agent sdk, supports tools to be provided directly in the MCP without client, but for the resoruces we still need to create the mcp client
#these client spawn the mcp servers and then get the resources
display(Markdown(await read_accounts_resource("Ed")))
display(Markdown(await read_strategy_resource("Ed")))

{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-09-08 11:47:50", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

#### **Trader Agent**

In [24]:
agent_name = "Ed"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

#fedding the data we got from the resources into the instructions 
instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

print(instructions)


You are a trader that manages a portfolio of shares. Your name is Ed and your account is under your name, Ed.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-09-08 11:47:50", 10000.0], ["2025-09-08 11:48:53", 10000.0], ["2025-09-08 11:49:03", 10000.0], ["2025-09-08 11:49:32", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have too

In [31]:
#lets run the trader 

for server in mcp_servers:  #connecting to all the mcp servers
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers) #initializing the Research Agent as a tool with its own MCP server with researching tools


#here we are initializing the trader agent with a research agent with its mcp as a tool and its now mcp in the mcp section
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool], #researcher agent as a tool with its mcp 
    mcp_servers=trader_mcp_servers, #providing its own mcp
    model="gpt-4o-mini",
)

resoruces = await read_accounts_resource(agent_name)
print(f'Current Resources: {resoruces}')

with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

resoruces = await read_accounts_resource(agent_name)
print(f'After Resources: {resoruces}')

Current Resources: {"name": "ed", "balance": 6229.8748, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"CRM": 10, "RH": 5}, "transactions": [{"symbol": "CRM", "quantity": 10, "price": 251.26152, "timestamp": "2025-09-08 11:54:11", "rationale": "Salesforce has strong AI business prospects and recent analyst attention. High volatility expected following earnings report."}, {"symbol": "RH", "quantity": 5, "price": 251.502, "timestamp": "2025-09-08 11:54:11", "rationale": "Luxury sector has potential upside. Recent earnings report could create price movement."}], "portfolio_value_time_series": [["2025-09-08 11:47:50", 10000.0], ["2025-09-08 11:48:53", 10000.0], ["2025-09-08 11:49:03", 10000.0], ["2025-09-08 11:49:32", 10000.0], ["2025-09-08 11:53:49", 10000.0], ["2025-09-08 11:54:11", 9994.9848], ["2025-09-08 11:54:11", 9992.4748], ["2025-09-08 11:54:19", 9992.4748], ["2025-09-08 12:03:05", 9992.4748], ["2025-09

### Summary of Actions Taken:

1. **Market Investigation**:
   - Reviewed current market conditions showing caution amid inflation concerns and a weakening labor market.
   - Key trending stocks identified: **Robinhood Markets (HOOD)**, **Opendoor Technologies (OPEN)**, and **SoFi Technologies (SOFI)**.

2. **Stock Price Checks**:
   - **Robinhood (HOOD)**: $101.25
   - **Opendoor (OPEN)**: $6.65
   - **SoFi (SOFI)**: $25.60

3. **Trades Executed**:
   - **Bought 10 shares of Robinhood (HOOD)** at $101.25 each.
   - **Bought 100 shares of Opendoor (OPEN)** at $6.65 each.
   - **Total Spending**: 
     - HOOD: $1,012.50
     - OPEN: $665.00

4. **Current Holdings**:
   - Existing positions held in stocks: 
     - **CRM**: 10 shares
     - **RH**: 5 shares 
     - **HOOD**: 20 shares 
     - **OPEN**: 200 shares
     - **SOFI**: 100 shares

5. **Balance Post-Trades**:
   - **Remaining Cash Balance**: Approximately $303.04.

### Next Steps:
- Monitor market conditions closely, and consider further trades if new opportunities arise based on news and stock performance.
- Keep an eye on the performance of the newly acquired stocks to capitalize swiftly on any favorable moves.

After Resources: {"name": "ed", "balance": 303.04479999999944, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"CRM": 10, "RH": 5, "HOOD": 20, "OPEN": 200, "SOFI": 100}, "transactions": [{"symbol": "CRM", "quantity": 10, "price": 251.26152, "timestamp": "2025-09-08 11:54:11", "rationale": "Salesforce has strong AI business prospects and recent analyst attention. High volatility expected following earnings report."}, {"symbol": "RH", "quantity": 5, "price": 251.502, "timestamp": "2025-09-08 11:54:11", "rationale": "Luxury sector has potential upside. Recent earnings report could create price movement."}, {"symbol": "HOOD", "quantity": 10, "price": 101.4525, "timestamp": "2025-09-08 12:04:05", "rationale": "Current interest in Robinhood due to market volatility and its position among trending stocks."}, {"symbol": "OPEN", "quantity": 100, "price": 6.6633000000000004, "timestamp": "2025-09-08 12:04:05", "rationa

#### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified.

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [10]:
from traders import Trader

trader = Trader("Abhinav")

In [11]:
# Create memory directory if it doesn't exist
import os
os.makedirs("memory", exist_ok=True)

# Now run the trader
await trader.run()

## Introducing our FOUR traders

We have four traders, and they are inspired by 4 titans of the industry.  

**Warren**, in homage to Warren Buffett  
**George**, in homage to George Soros  
**Ray**, in homage to Ray Dalio  
**Cathie**, in homage to Cathie Wood

They have initial investment strategies that are inspired by their namesakes; but they have the autonomy to change their own strategy over time if they wish, using a tool.


Check out the file `reset.py` to read their initial investment theses.

In [3]:
# Now, set our traders to the starting point - uncomment the line to reset..

from reset import reset_traders
reset_traders()

## Revealing the final changes

Some new MCP servers - memory and push notification - take a look at `mcp_params.py` and `push_server.py`

And this is a cool thing:

OpenAI Agents SDK has a nice feature that you can integrate with their Tracing code, so that you can monitor Trace messages in code.

See `tracers.py` - I've written a custom tracer that records our Trader activity and stores it in the database, so we can surface the Traders' inner thoughts on our UI.

#### AND NOW.. Reveal part 1

First, there are some settings that you can optionally add to your .env file:

`RUN_EVERY_N_MINUTES=60`  
This determines how often the trader agents are run, and it defaults to every 60 minutes if not specified in your .env file.

`RUN_EVEN_WHEN_MARKET_IS_CLOSED=False`  
This determines if the traders should still run out of hours, and defaults to False if not specified in your .env file.

`USE_MANY_MODELS=False`  
This determines whether to use DeepSeek, Gemini and Grok in addition to OpenAI, using DeepSeek API and OpenRouter.  
It defaults to False if not specified in your .env file, in which case gpt-4o-mini is used throughout.

Please make those changes if you wish!

Then take a look at the UI code in `app.py`

Then, open a new terminal (Shift + Ctrl + Backtick)

Change to this directory:  
`cd 6_mcp`

And run:  
`uv run app.py`

And take joy in the user interface!

### Now go and look at the engine:

`trading_floor.py`

It has the super-simple loop that's where the magic happens:


```
while True:
    await asyncio.gather(*[trader.run() for trader in traders])
    await asyncio.sleep(RUN_EVERY_N_MINUTES * 60)
```

You'll also see how it looks for the environment variables.

And finally, open a new terminal (Shift + Ctrl + Backtick, or press the Plus on the top right of the Terminals below)

Change to this directory:  
`cd 6_mcp`

And run:  
`uv run trading_floor.py`

And watch your user interface - see your trading team in action!